
# Boosted ECCT: Training Notebook
**Purpose:** Train ECCT baseline models for use in the Boosting Notebook.

**Instructions:** Switch the Colab runtime to GPU if possible (otherwise, switch 'device' in CONFIG to 'cpu'). Edit the CONFIG cell below before running any cells to select the code (LDPC or POLAR), blocklength, and other hyperparameters. Then run the notebook from top to bottom.


In [ ]:
# CONFIG --- edit these values before running ---

from datetime import datetime
RUN_TIMESTAMP = datetime.now().strftime("%d%b%Y_%H%M")

# Define CONFIG with fixed and independent values
CONFIG = {
    'code_type': 'LDPC',
    'code_n': 121,
    'code_k': 60,

    'ecct_repo': 'https://github.com/yoniLc/ECCT.git',
    'boost_repo': 'https://github.com/ghy1228/LDPC_Error_Floor.git',

    'epochs_base': 50,
    'epochs_post': 20,
    'target_uc': 1000,
    'uc_snr_db': 4.5,
    'eval_ebno_list': [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0],
    'batch_size': 128,
    'uc_batch_size': 512,
    'post_batch_size': 64,
    'seed': 42,
    'device': 'cuda'
}

# Calculate 'workdir' after 'code_type', 'code_n', and 'code_k' are defined
# Workdir should be unique per experiment to avoid UC mixing
CONFIG['workdir'] = f"/content/{CONFIG['code_type']}_n{CONFIG['code_n']}_k{CONFIG['code_k']}_{RUN_TIMESTAMP}"

print("CONFIG:", CONFIG)

## Setup: create workdir, clone repos, and install Python dependencies

In [ ]:

import os, sys, subprocess
workdir = CONFIG['workdir']
os.makedirs(workdir, exist_ok=True)
print('Workdir:', workdir)
# Clone repos if not already present
os.chdir(workdir)
if not os.path.isdir('ECCT'):
    print('Cloning ECCT...')
    subprocess.run(['git', 'clone', CONFIG['ecct_repo'], 'ECCT'], check=False)
else:
    print('ECCT already present.')
if not os.path.isdir('LDPC_Error_Floor'):
    print('Cloning LDPC_Error_Floor...')
    subprocess.run(['git', 'clone', CONFIG['boost_repo'], 'LDPC_Error_Floor'], check=False)
else:
    print('LDPC_Error_Floor already present.')

# Install common Python packages used by the ECCT repo
print('Installing Python packages (may take a minute)...')
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'einops', 'tqdm', 'matplotlib', 'scikit-learn'], check=False)

# If ECCT has requirements.txt, install it (safe guard)
req_path = os.path.join(workdir, 'ECCT', 'requirements.txt')
if os.path.exists(req_path):
    print('Installing ECCT/requirements.txt...')
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', req_path], check=False)
else:
    print('No ECCT requirements.txt detected.')

print('Setup done.')


In [ ]:
## Initial Setup and Timestamp Generation 🕐

import torch, glob
from torch.utils.data import DataLoader
from pathlib import Path
from datetime import datetime
import shutil

from google.colab import drive
drive.mount('/content/drive')

drive_base_folder = "/content/drive/MyDrive/Colab_Models"

# Use the SAME timestamp across cells
drive_destination_folder = os.path.join(
    drive_base_folder,
    f"{CONFIG['code_type']}_n{CONFIG['code_n']}_k{CONFIG['code_k']}_{RUN_TIMESTAMP}_ECCT_Boosted_Results"
)

os.makedirs(drive_destination_folder, exist_ok=True)
print("Saving results to:", drive_destination_folder)


## Sanity check: show key ECCT files (Main.py, Model.py, Codes.py)

In [ ]:

# List key files and print top of Main.py to confirm flags
os.chdir(CONFIG['workdir'] + '/ECCT')
print('Current dir:', os.getcwd())
!ls -la Main.py Model.py Codes.py || true
print('\n--- Main.py head ---\n')
!sed -n '1,240p' Main.py


## Baseline: Train ECCT (using Main.py)
This will run the repository's training entrypoint. Edit the command-line args below if you want different settings.

In [ ]:

# Run ECCT baseline via Main.py
os.chdir(os.path.join(CONFIG['workdir'], 'ECCT'))

cli = [
    sys.executable, 'Main.py',
    f"--gpus=0",
    f"--N_dec=6",
    f"--d_model=32",
    f"--code_type={CONFIG['code_type']}",
    f"--code_n={CONFIG['code_n']}",
    f"--code_k={CONFIG['code_k']}",
    f"--epochs={CONFIG['epochs_base']}",
    f"--batch_size={CONFIG['batch_size']}",
    f"--test_batch_size=2048",
    f"--lr=1e-4",
    f"--seed={CONFIG['seed']}"
]
print('Running:', ' '.join(cli))
subprocess.run(cli, check=False)
print('ECCT Main.py finished (or exited). Check Results_ECCT for outputs.')


## Helper: locate the saved baseline model (Main.py saves `best_model` under Results_ECCT)

In [ ]:
# --- Helper: Locate latest Results_ECCT directory under the current run workdir ---

results_root = os.path.join(CONFIG['workdir'], "ECCT", "Results_ECCT")

if not os.path.isdir(results_root):
    raise RuntimeError(
        f"Results_ECCT directory not found at {results_root}. "
        "You must run Main.py (baseline ECCT training) first."
    )

res_dirs = sorted(
    glob.glob(os.path.join(results_root, "*")),
    key=os.path.getmtime,
    reverse=True
)

if len(res_dirs) == 0:
    raise RuntimeError(
        f"No timestamped folders found in {results_root}. "
        "Did Main.py create any results?"
    )

model_dir = res_dirs[0]
model_path = os.path.join(model_dir, "best_model")

print("Using latest model directory:", model_dir)
print("Model path:", model_path, "Exists?", os.path.exists(model_path))

if not os.path.exists(model_path):
    raise RuntimeError(
        f"best_model not found in {model_dir}. "
        "Ensure Main.py completed successfully."
    )

model_dir  # Show path for notebook cell


In [ ]:
if 'model_dir' not in globals():
    raise RuntimeError("model_dir not defined — run training first.")

source_dir = model_dir

# Use the SAME folder as above
final_destination = os.path.join(
    drive_destination_folder,
    os.path.basename(source_dir)
)

try:
    shutil.copytree(source_dir, final_destination, dirs_exist_ok=True)
    print(f"Copied -> {final_destination}")
except Exception as e:
    print("Copy error:", e)
